In [33]:
#边界框
import imgaug as ia
import imgaug.augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
import time
import cv2

from PIL import Image
import os
import sys
import random
import imageio

#图像归一化
ia.seed(1)
# image = ia.quokka(size=(256, 256))
# bbs = BoundingBoxesOnImage([
#     BoundingBox(x1=65, y1=100, x2=200, y2=150),
#     BoundingBox(x1=150, y1=80, x2=200, y2=130)
# ], shape=image.shape)

#aug1=iaa.Affine(rotate=45)#右旋转45度
#seq = iaa.Sequential([
aug1=iaa.AdditiveGaussianNoise(scale=0.2*255)#加噪声
aug2=iaa.GaussianBlur(sigma=(0.0, 3.0)) #变模糊
aug3=iaa.AllChannelsCLAHE(clip_limit=(1,10))#图像增强
aug4=iaa.Affine(rotate=-45)#左旋转45度
aug5=iaa.Affine(rotate=45)#右旋转45度
aug6=iaa.Affine(scale=(0.4, 0.7))#Scale images to a value of 50 to 150% of their original size变大
aug7=iaa.Affine(scale=(1.3, 1.6))#Scale images to a value of 50 to 150% of their original size#缩小
aug8=iaa.GammaContrast((2.5, 2.5), per_channel=True)#色相变暗
aug9=iaa.Grayscale(alpha=(0.9))#色相变灰
aug10=iaa.PiecewiseAffine(scale=(0.08))#扭曲图像

# image1,bbs1=aug1(image=image, bounding_boxes=bbs)
# image2,bbs2=aug2(image=image, bounding_boxes=bbs)

# image = Image.open("G:/test-data/1.jpg").convert("RGB")

def boxFromLine(line):
    line = line.strip()
    listFromLine = line.split(' ')        
    cx = float(listFromLine[1]) * ww
    cy = float(listFromLine[2]) * hh
    w = float(listFromLine[3]) * ww
    h = float(listFromLine[4]) * hh
    x1 = cx - w / 2
    y1 = cy - h / 2
    x2 = cx + w / 2
    y2 = cy + h / 2
    return BoundingBox(x1, y1, x2, y2)

def linesFromBoxs(boxs, ww, hh):
    lines = []
    for i in range(boxs.shape[0]):
        x1 = boxs[i][0]
        y1 = boxs[i][1]
        x2 = boxs[i][2]
        y2 = boxs[i][3]
        cx = (x1 + x2) / 2
        cy = (y1 + y2) / 2
        w = x2- x1
        h = y2 - y1
        line = "0 " + '%.6f'%(cx / ww) + " " + '%.6f'%(cy / hh) + " " + '%.6f'%(w / ww) + " " + '%.6f'%(h / hh)
        lines.append(line)
    return lines
    

imgFolder = "G:/detect-train/Train-WhiteCell/data/1/"
imgFolderDst = "G:/detect-train/Train-WhiteCell/data/2/"
imgsT = os.listdir(imgFolder)
imgs = []
for file in imgsT:
    if file.endswith(".jpg") or file.endswith(".bmp"):
        file2 = file.replace(".jpg", ".txt")
        file2 = file2.replace(".bmp", ".txt")
        imgInfo = (os.path.join(imgFolder, file), os.path.join(imgFolder, file2))                
        imgs.append(imgInfo)

countImg = 0
for i in range(len(imgs)):
    imgfile = imgs[i][0]
    txtfile = imgs[i][1]
    image = imageio.imread(imgfile)
    ww = image.shape[0]
    hh = image.shape[1]
    print(image.shape)
    print(imgfile)
    print(txtfile)
    fh = open(txtfile)
    lines = fh.readlines()
    fh.close()   
    bboxs = []
    for line in lines:        
        bboxs.append(boxFromLine(line));
    bbs = BoundingBoxesOnImage(bboxs, image.shape)
    for j in range(5):
        countImg+=1
        imgsave = os.path.join(imgFolderDst, str(countImg) + ".jpg")
        txtsave = os.path.join(imgFolderDst, str(countImg) + ".txt")
        print(txtsave)
        
        image2, bbs2 = aug2(image=image, bounding_boxes=bbs)
        boxs = bbs2.to_xyxy_array()    
        lines = linesFromBoxs(boxs, ww, hh)
        fh = open(txtsave, 'w')   
        print(lines)
        fh.writelines(lines)
        fh.close()         
        imageio.imwrite(imgsave, image2)
    




(1024, 1224, 3)
G:/detect-train/Train-WhiteCell/data/1/_c_23_99_1.jpg
G:/detect-train/Train-WhiteCell/data/1/_c_23_99_1.txt
G:/detect-train/Train-WhiteCell/data/2/1.txt
['0 0.586719 0.412500 0.050000 0.063889']
G:/detect-train/Train-WhiteCell/data/2/2.txt
['0 0.586719 0.412500 0.050000 0.063889']
G:/detect-train/Train-WhiteCell/data/2/3.txt
['0 0.586719 0.412500 0.050000 0.063889']
G:/detect-train/Train-WhiteCell/data/2/4.txt
['0 0.586719 0.412500 0.050000 0.063889']
G:/detect-train/Train-WhiteCell/data/2/5.txt
['0 0.586719 0.412500 0.050000 0.063889']
(1024, 1224, 3)
G:/detect-train/Train-WhiteCell/data/1/_c_24_2_1.jpg
G:/detect-train/Train-WhiteCell/data/1/_c_24_2_1.txt
G:/detect-train/Train-WhiteCell/data/2/6.txt
['0 0.319141 0.562500 0.044531 0.052778', '0 0.511328 0.521528 0.038281 0.054167', '0 0.050000 0.931250 0.040625 0.059722']
G:/detect-train/Train-WhiteCell/data/2/7.txt
['0 0.319141 0.562500 0.044531 0.052778', '0 0.511328 0.521528 0.038281 0.054167', '0 0.050000 0.931250 0

In [6]:
file = "fdd.jpg"
file2 = file.replace(".bmp", ".txt")
print(file2)
fff = [1,2]
len(fff)

fdd.jpg


2

In [31]:
line = "23 56"
line = line.strip()
listFromLine = line.split(' ')
int(listFromLine[0])

'%.3f'%-3.14

'-3.140'